## Erstmal auf den Zyklus gruppieren und dann Features erstellen

### Import

In [1]:
import pandas as pd
import numpy as np

import os
import sys
import logging

import functions as fu
from feature_engineering import *

# Pfade setzen
c_fold = fu.get_folder()
p_fold = c_fold + '\data\Prepared'
folder = c_fold + '\data\Processed\Zyklus'


# Initialize Logging
logging.basicConfig(format='[%(asctime)s] %(levelname)s: %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO,
                    handlers=[
                        logging.FileHandler(
                            filename='datalogger.log',
                            mode='a',
                            encoding='utf-8'),
                        logging.StreamHandler(sys.stdout)])

In [31]:
def stark_leicht(type_, current_mean):
    if type_ == 'D': #discharge
        if current_mean >= 3:
            return "entladen stark"
        else:
            return "entladen leicht"
    if type_ == 'C': #charge
        if current_mean <= -3:
            return "laden stark"
        else:
            return "laden leicht"
    if type_ == 'R':
        return "pause"
    
        

# for file in os.listdir(p_fold):
#     if ".csv" in file:
file = "prepared_RW1.csv"
logging.info(f"<<< Started processing file: {file} >>>")
new_filename = file.replace("prepared","processed")

df = pd.read_csv(f"{p_fold}\\{file}", index_col = 0,
                         #nrows= 100000
                        )
logging.info(f"<<< Size of Dataframe: {df.shape} >>>")

#Auf Ebene Batterie + Zyklus gruppieren mit benötigten Aggregationen
df_pv = pd.pivot_table(
   df,
   index=['batteryname','zyklus','comment','type'],
   aggfunc={'time': np.min, 'relativeTime': np.max, 'voltage': [np.min, np.max, np.mean], 'current': [np.min, np.max, np.mean], 'temperature': [np.min, np.max, np.mean]}
).reset_index()

df_pv.columns = ['_'.join(col) for col in list(df_pv.columns)]



df_pv['ladearten'] = df_pv.apply(lambda x: stark_leicht(x['type_'],x['current_mean']),axis=1)


logging.info(f"<<< Start saving dataframe with shape {df_pv.shape} >>>")
df_pv.to_csv(f"{folder}\\{new_filename}", sep=',', index=False)
logging.info(f"<<< Saved processed file in directory {folder}\\{new_filename} >>>")

[08:25:05] INFO: <<< Started processing file: prepared_RW1.csv >>>
[08:25:10] INFO: <<< Size of Dataframe: (2958840, 11) >>>
[08:25:11] INFO: <<< Start saving dataframe with shape (17065, 16) >>>
[08:25:11] INFO: <<< Saved processed file in directory C:\Users\sebis\OneDrive\Desktop\FOM\3. Semester\Big Data Analyseprojekt\battery_failure_predictions\data\Processed\Zyklus\processed_RW1.csv >>>


In [32]:
df = pd.read_csv(f"{folder}\\{new_filename}")

In [33]:
df

,batteryname_,zyklus_,comment_,type_,current_amax,current_amin,current_mean,relativeTime_amax,temperature_amax,temperature_amin,temperature_mean,time_amin,voltage_amax,voltage_amin,voltage_mean,ladearten
0,RW1,0,low current discharge at 0.04A,D,0.041,0.018,0.039910,188976.68,18.79972,17.08313,17.939268,1538.07,4.196,3.200,3.850859,entladen leicht
1,RW1,1,low current charge,C,-0.015,-0.041,-0.039912,191271.64,18.64223,17.06738,17.867832,190514.75,4.200,3.209,3.881712,laden leicht
2,RW1,2,reference charge,C,-0.002,-0.041,-0.022038,3446.64,18.37451,18.09103,18.232178,381786.29,4.200,4.192,4.199977,laden leicht
3,RW1,3,reference discharge,D,1.003,0.999,0.999929,7201.82,23.56742,18.39025,21.798768,385232.92,4.009,3.200,3.636954,entladen leicht
4,RW1,4,reference charge,C,-0.010,-2.008,-0.551101,13098.45,25.28264,17.69732,20.055134,392434.74,4.200,3.769,4.193979,laden leicht
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17060,RW1,17060,discharge (random walk),D,1.005,0.999,0.999910,300.00,27.99582,27.13821,27.568021,13322445.61,4.079,3.841,3.906276,entladen leicht
17061,RW1,17061,rest (random walk),R,0.000,0.000,0.000000,1.11,27.99582,27.99582,27.995820,13322745.58,4.004,3.945,3.974500,pause
17062,RW1,17062,discharge (random walk),D,0.502,0.499,0.499874,300.00,28.13615,27.98022,28.068808,13322746.72,3.945,3.927,3.939203,entladen leicht
17063,RW1,17063,rest (random walk),R,0.000,0.000,0.000000,0.20,28.08937,28.08937,28.089370,13323046.69,4.002,3.981,3.991500,pause
